In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

Importamos los datos

In [2]:
dataframe = pd.read_csv("data.csv")
print(dataframe)

        Periodo      ID Especialidad del momento Codigo curso  \
0      201220.0   135.0                     INGE      ICE3210   
1      201220.0   135.0                     INGE      ICE3210   
2      201220.0   135.0                     INGE      ING2152   
3      201220.0   135.0                     INGE      ING2152   
4      201220.0   135.0                     INGE      ING3520   
5      201220.0   135.0                     INGE      ING3520   
6      201220.0   135.0                     INGE      IOC3150   
7      201220.0   135.0                     INGE      IOC3150   
8      201220.0   916.0                     INGO      CEG2006   
9      201220.0   916.0                     INGO      CEG2006   
10     201220.0   916.0                     INGO      ING5230   
11     201220.0   916.0                     INGO      ING5230   
12     201220.0   916.0                     INGO      ING5411   
13     201220.0   916.0                     INGO      ING5411   
14     201220.0   916.0  

Filtramos las filas que tengan notas nulas y los datos duplicados.  
Normalizamos la nota a tras transformarla en decimal eliminando ademas las filas que no tengan una nota numérica, y convertimos el semestre a string dado a que no nos importa su valor numérico.

In [3]:
dataframe = dataframe[pd.to_numeric(dataframe['Nota'], errors='coerce').notnull()]
normalize = lambda x : (x-1)/6
dataframe['Nota'] = dataframe['Nota'].apply(float)
dataframe['Nota'] = dataframe['Nota'].apply(normalize)
dataframe = dataframe.drop_duplicates(subset=['Periodo', 'ID', 'Codigo curso'])
dataframe['Periodo'] = dataframe['Periodo'].astype(int).apply(str)
dataframe['ID'] = dataframe['ID'].astype(int)
print(dataframe)

      Periodo       ID Especialidad del momento Codigo curso  \
0      201220      135                     INGE      ICE3210   
2      201220      135                     INGE      ING2152   
4      201220      135                     INGE      ING3520   
6      201220      135                     INGE      IOC3150   
8      201220      916                     INGO      CEG2006   
10     201220      916                     INGO      ING5230   
12     201220      916                     INGO      ING5411   
14     201220      916                     INGO      ING5510   
16     201220      916                     INGO      ING6410   
18     201220      916                     INGO      IOC4110   
20     201220      916                     INGO      IOC4140   
22     201220     2141                      NaN      ING6110   
24     201220     2141                      NaN      ING6210   
26     201220      816                     INGO      CEG1010   
28     201220      816                  

Filtramos los alumnos sin especialidad o con especialidad nula pues no nos interesan para entrenamiento, validación o testeo, y eliminamos la columna con el nombre del ramo.  
Guardamos los ramos realizados por alumnos sin especialidad, para luego eliminar las filas que tengan ramos fuera de este conjunto.

In [4]:
dataframe = dataframe[pd.notnull(dataframe['Especialidad del momento'])]
dataframe = dataframe[(dataframe['Especialidad del momento'] != 'ING')]
dataframe = dataframe[(dataframe['Especialidad del momento'] != 'INGA')]
dataframe = dataframe.drop(['Nombre curso'],axis=1).reset_index(drop=True)
print (dataframe)


      Periodo       ID Especialidad del momento Codigo curso      Nota
0      201220      135                     INGE      ICE3210  0.683333
1      201220      135                     INGE      ING2152  0.766667
2      201220      135                     INGE      ING3520  0.750000
3      201220      135                     INGE      IOC3150  0.700000
4      201220      916                     INGO      CEG2006  0.850000
5      201220      916                     INGO      ING5230  0.683333
6      201220      916                     INGO      ING5411  0.783333
7      201220      916                     INGO      ING5510  0.816667
8      201220      916                     INGO      ING6410  0.800000
9      201220      916                     INGO      IOC4110  0.733333
10     201220      916                     INGO      IOC4140  0.716667
11     201220      816                     INGO      CEG1010  0.733333
12     201220      816                     INGO      FRM1100  0.550000
13    

Asignamos indentificadores para todos los campos no numéricos(aparte del label), y luego los transformamos en campos numericos.

In [5]:
periods = list(dataframe['Periodo'].unique())
periods.insert(0, 'dummy')
courses = list(dataframe['Codigo curso'].unique())
courses.insert(0, 'dummy')
dataframe['Periodo'] = dataframe['Periodo'].apply(periods.index)
dataframe['Codigo curso'] = dataframe['Codigo curso'].apply(courses.index)

Obtenemos una lista con la cantidad de alumnos por especialidad, luego obtenemos la especialidad con menos cantidad de alumnos y luego balanceamos eliminando alumnos al azar de las otras especialidades hasta que queden de igual cantidad.

In [6]:
studentsframe = dataframe.drop_duplicates(subset=['ID']).drop(columns=['Periodo','Codigo curso', 'Nota'])
count_list = studentsframe.drop(columns=['ID']).groupby(['Especialidad del momento']).size().reset_index(name='count').values.tolist()
min_value = min(count_list, key=lambda x: x[1])[1]
for spec in count_list:
    ids = studentsframe.loc[studentsframe['Especialidad del momento'] == spec[0]].sample(spec[1]- min_value).index.values.tolist()
    studentsframe = studentsframe.drop(ids, axis=0)
students = studentsframe.values.tolist()

Obtenemos cual es el maximo de ramos en un semestre para un alumno, la maxima cantidad de semestres que tiene un alumno y luego generamos el dataframe de resultado.

In [7]:
periods = dataframe.drop_duplicates(subset=['Periodo', 'ID']).drop(columns=['Periodo', 'Especialidad del momento', 'Codigo curso', 'Nota']).groupby(['ID']).size().reset_index(name='count').values.tolist()
max_periods = max(periods, key=lambda x: x[1])[1]
courses_in_a_period = dataframe.drop(columns=['Especialidad del momento', 'Nota']).groupby(['ID','Periodo']).size().reset_index(name='count').values.tolist()
max_courses_in_a_period = max(courses_in_a_period, key=lambda x: x[2])[2]
finaldataframe = pd.DataFrame(columns=['ID','spec'])

for p in range(0, max_periods):
    finaldataframe['P'+str(p)] = 0
    for c in range(0, max_courses_in_a_period):
        finaldataframe['P'+str(p)+'C'+str(c)] = 0
        finaldataframe['P'+str(p)+'C'+str(c)+'N'] = 0
for s in students:
    array = [s[0],s[1]]
    p_count = 0
    for p in dataframe.loc[dataframe['ID']==s[0]]['Periodo'].unique():
        array.append(p)
        c_count = 0
        for c in dataframe.loc[dataframe['ID']==s[0]].loc[dataframe['Periodo']==p].drop_duplicates(subset=['Codigo curso']).drop(columns=['Periodo','ID', 'Especialidad del momento']).values.tolist():
            array.append(c[0])
            array.append(c[1])
            c_count = c_count + 1
        for c in range(0, max_courses_in_a_period - c_count):
            array.append(0)
            array.append(0)
        p_count = p_count + 1
    for p in range(0, max_periods - p_count):
        array.append(0)
        for c in range(0, max_courses_in_a_period):
            array.append(0)
            array.append(0)
    finaldataframe.loc[-1] = array
    finaldataframe.index =  finaldataframe.index + 1
print(finaldataframe)

           ID  spec  P0   P0C0     P0C0N   P0C1     P0C1N  P0C2     P0C2N  \
307       916  INGO   1    5.0  0.850000    6.0  0.683333   7.0  0.783333   
306  11881373  INGE   1   22.0  0.633333   24.0  0.300000  25.0  0.666667   
305       568  INGO   1   65.0  0.566667    3.0  0.550000  68.0  0.683333   
304       693  INGE   1   69.0  0.216667   70.0  0.450000  71.0  0.566667   
303  17170244  INGE   1   23.0  0.583333   24.0  0.583333  70.0  0.650000   
302       524  INGE   1   22.0  0.800000   23.0  0.733333  70.0  0.066667   
301       292  INGO   1   33.0  0.500000    4.0  0.533333  11.0  0.683333   
300  19372471  INGE   1   23.0  0.500000   24.0  0.516667  70.0  0.500000   
299       482  INGI   1    1.0  0.650000   38.0  0.833333  54.0  0.783333   
298       258  INGI   1    3.0  0.533333   55.0  0.500000  46.0  0.683333   
297  19566059  INGI   1   52.0  0.800000   53.0  0.783333   9.0  0.716667   
296       187  INGC   1   83.0  0.800000   86.0  0.516667  87.0  0.750000   

Exportamos el csv con todos los casos.

In [8]:
finaldataframe.to_csv("allcases.csv")

Generamos los casos de entrenamiento, prueba y validación.

In [9]:
dataframes_per_specialization = []
for spec in count_list:
    dataframes_per_specialization.append(finaldataframe.loc[finaldataframe['spec']==spec[0]])

training_array = []
testing_array = []
validation_array = []
for dps in dataframes_per_specialization:
    length = len(dps.index)
    validation_subset = dps.sample(int(length*0.1))
    aux = dps.drop(validation_subset.index)
    testing_subset = aux.sample(int(length*0.2))
    aux = aux.drop(testing_subset.index)
    training_array.append(aux)
    testing_array.append(testing_subset)
    validation_array.append(validation_subset)

training = pd.concat(training_array).sample(frac=1).reset_index(drop=True)
testing = pd.concat(testing_array).sample(frac=1).reset_index(drop=True)
validation = pd.concat(validation_array).sample(frac=1).reset_index(drop=True)

Finalmente exportamos los csv's

In [10]:
training.to_csv("training.csv", index=False)
testing.to_csv("testing.csv", index=False)
validation.to_csv("validation.csv", index=False)